In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import csv
import random
import matplotlib.pyplot as plt
import statistics 
from scipy import stats
import io

plt.rcParams["figure.figsize"] = (10,10)



### *Importing files from github for google colab


In [8]:
url = 'https://raw.githubusercontent.com/sobhan-moosavi/varicella_study/main/contact_rules.csv'
rules_df  = pd.read_csv(url, sep='delimiter',header=None)
url2='https://raw.githubusercontent.com/sobhan-moosavi/varicella_study/main/frequency_contact.csv'
frequency_df= pd.read_csv(url2, sep='delimiter',header=None)
url3='https://raw.githubusercontent.com/sobhan-moosavi/varicella_study/main/varicella_prevalance.csv'
V_prevalence_df =pd.read_csv(url3, sep='delimiter',header=None)
url4='https://raw.githubusercontent.com/sobhan-moosavi/varicella_study/main/population.csv'
population_df=pd.read_csv(url4, sep='delimiter',header=None)

<ipython-input-8-f3d9925c4710>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  rules_df  = pd.read_csv(url, sep='delimiter',header=None)
<ipython-input-8-f3d9925c4710>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  frequency_df= pd.read_csv(url2, sep='delimiter',header=None)
<ipython-input-8-f3d9925c4710>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  V_prevalence_df =pd.read_csv(url3, sep='delimiter',heade

### *importing files from pc


# THE GRAPH CONTROL PANEL

In [3]:
N =300  # Number of nodes
setting='home'    #the setting we are evaluating(home,work,school,other)
D=10#number of days the model runs
increase_rate=.5 #the rate of increasing weight per contact
lower_weight=0  #the minimum weight between two node
threshold_n=1     #threshold of product weight two nodes with same neighbor for increasing weight
increse_rate_n=0  #increse rate weight for two nodes with same neighbor

## 1-Defining Nodes

In [4]:
G = nx.DiGraph()

In [5]:
#defining a dictionary of nodes
available_groups  = [] #16_age_groups
group_popularity = [] #based on consensus year 95
v_prevalence=dict() #based on meta analysis(data is not exact!! update needed!!)contan(age_group,prevalance,lower CL)
nodes_dic=dict()


for i, row in population_df.iterrows():
    available_groups.append(row[0])
    group_popularity.append(float(row[1]))
    

for i, row in V_prevalence_df.iterrows():
    v_prevalence[row[0]]=[float(row[1]),float(row[2])]
                    
for i in range(N):
    while True:
        r=random.random()
        t=random.randint(0,15)
        if r<=group_popularity[t]:
            nodes_dic[f'n{i+1}']={'age':available_groups[t]}
            break


for i in range(N):
    a=v_prevalence[nodes_dic[f'n{i+1}']['age']][0]
    b=v_prevalence[nodes_dic[f'n{i+1}']['age']][1]
    r=np.random.random()
    t=np.random.normal(a,(a-b)) 
    if r<=t:
        nodes_dic[f'n{i+1}']['infection']=1
    else:
        nodes_dic[f'n{i+1}']['infection']=0
        

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
#importing data for frequency of contact in each setting for comparison
frequency=dict()

for i, row in frequency_df.iterrows():
    frequency[row[0]]=dict()
    for i in range(1,6):
        frequency[row[0]][i]=float(row[i])

### Which `group` each node belongs to?

* We  used `Age specific population for Iran` to define probability of each group

In [ ]:

# let's see the number of nodes in each group:

age_count = {}
for n in nodes_dic.keys():
    
    if nodes_dic[n]['age'] in age_count:
        age_count[nodes_dic[n]['age']] = age_count[nodes_dic[n]['age']] + 1
    else:
        age_count[nodes_dic[n]['age']] = 1
age_count

## 2-Defining Connection Rules between Node Groups

### `We define rules in terms of connection probability between each two groups`


In [ ]:
#importing rules from .csv to nested dictionary
rules={}  # rules contain ->{location:{contactor:{contactee:contact number}
temp_0={}
temp_1={}
location=[]
contactor=[]
contactee=[]
contact_number=[]

for i, row in rules_df.iterrows():
    
    location.append(row[0])
    contactor.append(row[1])
    contactee.append(row[2])
    contact_number.append(float(row[3]))

for o in range (0,1280,256):
    for m in range(o,o+256,16):
        for n in range (m,m+16):
            
            temp_0[contactee[n]]=contact_number[n]
        temp_1[contactor[m]]=temp_0.copy()
    rules[location[o]]=temp_1.copy()     

## 3- Visualization for `SMALL` graph

### ⚠️ Only for SMALL graphs (i.e. noes < 100)

## 4-Defining Rules to Connecting Nodes and Adjusting `Probabilities/Distances`

* This would be the step that our Naive graph would evolve over the time through a simulation process
* we monitor connection between nodes and update weights based on connections

In [ ]:
#in this block we've defiend a function "Run" 
def Run(increase_rate,threshold_n,increse_rate_n,D):
    
    #building the graph based on nodes_dic
    
    for i in nodes_dic.keys():
        for j in nodes_dic.keys():
            if i!=j and rules[setting][nodes_dic[i]['age']][nodes_dic[j]["age"]]!=0:
               #this condition determine if a node is added to graph or not based on setting 
                G.add_edge(i,j,weight=1)
     #counting number of each age group in graph  
    group_count = {}
    for n in nx.nodes(G):
    
        if nodes_dic[n]['age'] in group_count:
            group_count[nodes_dic[n]['age']] = group_count[nodes_dic[n]['age']] + 1
        else:
            group_count[nodes_dic[n]['age']] = 1
            
            
    
    #bulding empty dictinary "contacted" for storing contacts in single day
    #building empty dictionary for monitoring all contacts during all days
    #building empty dictionary for monitoring all weights during all days
    contacted={}
    contacted_all={}
    weight_all={} 
    
    for i in nx.edges(G):
        
        if i[0] in contacted_all.keys():
            contacted[i[0]][i[1]]=0
            contacted_all[i[0]][i[1]]=[]
            weight_all[i[0]][i[1]]=[]
                 
        else :
            contacted[i[0]]={i[1]:0}
            contacted_all[i[0]]={i[1]:[]}
            weight_all[i[0]]={i[1]:[]}
            
        
        
            
    
    for d in range (0,D):#we are running the model for D days
        
        #building a dictionary that includes number of 'weight addition' and number of nodes viable for 'weight substraction'
        N_contacted=dict()
        for i in nx.nodes(G):
            N_contacted[i]={}
            for j in group_count.keys():
                N_contacted[i][j]={'N_base':0,'N_c':0,'decrease_rate':0 }
                               
        #iterating thorough the graph and storing connections in 'contacted'
        for i in nx.edges(G):
                                   
            age_p=rules[setting][nodes_dic[i[0]]['age']][nodes_dic[i[1]]["age"]]
            r=random.random()
            
            if r <(age_p/group_count[nodes_dic[i[1]]['age']])*G.edges[i[0],i[1]]['weight']:
                contacted[i[0]][i[1]]=1
                contacted_all[i[0]][i[1]].append(1)#storing contacts of each run in this dic
              
            else:
                contacted[i[0]][i[1]]=0
                contacted_all[i[0]][i[1]].append(0)
                               
        # iterating thorough 'contacted' and updating the weights based on it
        for i in nx.edges(G):
            
            age_p=rules[setting][nodes_dic[i[0]]['age']][nodes_dic[i[1]]["age"]]
            
            upper_weight=1/age_p*(group_count[nodes_dic[i[1]]['age']]-age_p)
           
            if contacted[i[0]][i[1]]==1 and G[i[0]][i[1]]['weight']*(1+increase_rate)<upper_weight:
                #we don't want the weight to be more than 'upper weight'

                t=G[i[0]][i[1]]['weight']+increase_rate
                G.add_edge(i[0],i[1],weight=t)
                N_contacted[i[0]][nodes_dic[i[1]]['age']]['N_c']+=1
            if G[i[0]][i[1]]['weight']-N_contacted[i[0]][nodes_dic[i[1]]['age']]['decrease_rate']<lower_weight:
                N_contacted[i[0]][nodes_dic[i[1]]['age']]['N_base']+=1
         
        #we calculate how much weight is added overall,and then distrubt it between all nodes that should decrease weight
        for i in nx.edges(G):
            
            N_base= N_contacted[i[0]][nodes_dic[i[1]]['age']]['N_base']
            N_c=N_contacted[i[0]][nodes_dic[i[1]]['age']]['N_c']
            age_p=rules[setting][nodes_dic[i[0]]['age']][nodes_dic[i[1]]["age"]]
            pop=group_count[nodes_dic[i[1]]['age']]-N_base-N_c
            if pop!=0:
                N_contacted[i[0]][nodes_dic[i[1]]['age']]['decrease_rate']=(increase_rate*N_c)/pop
            else:
                N_contacted[i[0]][nodes_dic[i[1]]['age']]['decrease_rate']=0
            
            if contacted[i[0]][i[1]]==0 and (G[i[0]][i[1]]['weight']-N_contacted[i[0]][nodes_dic[i[1]]['age']]['decrease_rate'] >lower_weight):
                #we don't want the weight to be lower than lower_weight
                t=G[i[0]][i[1]]['weight']-N_contacted[i[0]][nodes_dic[i[1]]['age']]['decrease_rate']
                G.add_edge(i[0],i[1],weight=t)
            weight_all[i[0]][i[1]].append(G.edges[i[0],i[1]]['weight'])

        #updating weight between two nodes with same neighbor
        #for i in nodes_dic.keys():
            #for j in nodes_dic.keys():
                #if i!=j:
                    #for o in nodes_dic.keys():
                        #if o!=j and o!=i:

                            #if (G[i][o]['weight']*G[i][j]['weight'])>threshold_n and G[o][j]['weight']<1:
                                #t=G[o][j]['weight']+increse_rate_n
                                #G.add_edge(i,j,weight=t)
    return(contacted_all,G,weight_all,group_count)


### Visualize the graph after the updates

# 5- Monitroing the graph
### we monitor the graph for age specific number of contact per day and frequeny of contact

In [ ]:
#this function returns p_value wilcoxon for comparison age stratified number of contacts 
   

def metric_A_wilx(results) :   

    contacted_all=results[0]
    G=results[1]
    group_count=results[3]

    L1=[]
    L2=[]

    rules_m=dict()
    rules_m[setting]=dict()

    for i in rules[setting].keys():
        rules_m[setting][i]=dict()
        for j in rules[setting][i].keys():
             rules_m[setting][i][j]=0


    for i in nx.edges(G):
        t=statistics.mean(contacted_all[i[0]][i[1]])/(group_count[nodes_dic[i[0]]['age']])
        rules_m[setting][nodes_dic[i[0]]['age']][nodes_dic[i[1]]['age']]+=t
   
    for i in rules_m[setting].keys():
        for j in rules_m[setting][i].keys():
            L1.append(rules[setting][i][j])
            L2.append(rules_m[setting][i][j])

    I=stats.wilcoxon(L1,L2, zero_method='wilcox', correction=False, alternative='two-sided', mode='auto')
    return(I,rules_m)

In [ ]:
#this function returns the absoloute error for age stratified number of contact

def metric_A_abs(results):
    
    L=[]

    contacted_all=results[0]
    G=results[1]
    group_count=results[3]

    rules_m=dict()
    rules_m[setting]=dict()

    for i in rules[setting].keys():
        rules_m[setting][i]=dict()
        for j in rules[setting][i].keys():
             rules_m[setting][i][j]=0


    for i in nx.edges(G):
        
        t=statistics.mean(contacted_all[i[0]][i[1]])/(group_count[nodes_dic[i[0]]['age']])
        rules_m[setting][nodes_dic[i[0]]['age']][nodes_dic[i[1]]['age']]+=t

    for i in rules_m[setting].keys():
        for j in rules_m[setting][i].keys():

                L.append(abs((rules_m[setting][i][j])-(rules[setting][i][j])))

    I=statistics.mean(L)
    return(I,rules_m)

In [ ]:
#defining a function that returns mean absoloute error 'frequency of contacts ' in the last x day of model
def metric_F(results):
    
    n=0
    
    
    contacted_all=results[0]
    G=results[1]
    group_count=results[3]
    
    L1=[]
    L2=[]
    
    frequency_m=dict()
    for i in frequency.keys():
        frequency_m[i]={1:0,2:0,3:0,4:0,5:0}

    for i in contacted_all.keys():
        for j in contacted_all[i].keys():
            if contacted_all[i][j][-1]==1:
                
                age_p=rules[setting][nodes_dic[i]['age']][nodes_dic[j]["age"]]
        
                t=(age_p/group_count[nodes_dic[i]['age']])*G.edges[i,j]['weight']

                n+=1

                if sum(contacted_all[i][j])>1 and t<(1/30) :
                    frequency_m[setting][4]+=1

                elif t>=(1/30) and t<=(2/30):
                    frequency_m[setting][3]+=1

                elif t>(2/30) and t <=(2/7):
                    frequency_m[setting][2]+=1

                elif t>(2/7):

                    frequency_m[setting][1]+=1
                else:
                    frequency_m[setting][5]+=1

    for i in frequency_m[setting].keys():
        if n!=0:
        
            frequency_m[setting][i]=frequency_m[setting][i]/n
        if n==0:
            
            frequency_m[setting][i]=0
        
            
        
    
    for i in frequency_m[setting].keys():
        
        L1.append(frequency[setting][i])
        L2.append(frequency_m[setting][i])
        
    
    I=stats.wilcoxon(L1,L2, zero_method='wilcox', correction=False, alternative='two-sided', mode='auto')
    #print('for the first time:',frequency_m[setting][5],'\nless than once a month:',frequency_m[setting][4],'\nabout once or twice a month:',frequency_m[setting][3],'\nabout once or twice a week',frequency_m[setting][2],'\ndaily or almost daily',frequency_m[setting][1])

    return(I,frequency_m)

# evaluating results
### this part of code evaluates results of  function Run for the inputs

###### Run -> input : increase_rate, threshold_n, increse_rate_n, D 
###### Run -> output : contacted_all, G, weight_all, group_count


In [ ]:
#this part returns the absoloute error for age stratified number of contact
results=Run(increase_rate,threshold_n,increse_rate_n,D )
M_abs=metric_A_abs(results)
print(M_abs[0])

In [ ]:
#this part shows number of age stratified contact from data and model for comparision
for i in rules[setting].keys():
    for j in rules[setting].keys():
        print(i,'->',j,M_abs[1][setting][i][j],'//',rules[setting][i][j])

In [ ]:
#this part returns p_value wilcoxon for comparison age stratified number of contacts 

print('%f' % metric_A_wilx(results)[0][1])

In [ ]:
#this shows the mean of all edeges across the graph(it's supposed to stay close to one)
mean_weights=[]
for i in nx.edges(results[1]):
             
    mean_weights.append(results[1].edges[i[0],i[1]]['weight'])
            
print(statistics.mean(mean_weights))

In [ ]:
#this shows the fitting for frequency
results_F=metric_F(results)[1]

plt.rcParams["figure.figsize"] = (5,5)

for i in results_F[setting].keys():
    print (setting,':',i,'//',results_F[setting][i],'//',frequency[setting][i])
    plt.plot(i,results_F[setting][i],'+r',linewidth=1, markersize=12,alpha=1)
    plt.plot(i,frequency[setting][i],'+b',linewidth=1, markersize=12,alpha=1)

plt.show()


In [ ]:
#this figure shows evoloution of weights thorough time
plt.rcParams["figure.figsize"] = (10,10)

for i in nx.edges(results[1]):
    
    plt.plot(range(0,D),results[2][i[0]][i[1]],'k.',linewidth=.01, markersize=.5,alpha=.1)
plt.xlabel("days")
plt.ylabel('weight')
plt.show()


In [ ]:
weight_final={0:[],60:[],120:[],180:[],240:[],300:[],359:[]}
for v in weight_final.keys():
    for i in nx.edges(results[1]):
        
        weight_final[v].append(results[2][i[0]][i[1]][v])

for i in reversed(weight_final.keys()):
    plt.hist(weight_final[i],bins=20)
plt.axis([0,50,0,80000])
plt.xlabel("weight")
plt.ylabel('frequency')
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (5,5)
plt.hist(weight_final[359],bins=100)
plt.axis([0,50,0,20000])
plt.xlabel("weight")
plt.ylabel('frequency')
plt.show()
plt.hist(weight_final[359],bins=100)
plt.xlabel("weight")
plt.ylabel('frequency')
plt.axis([0,50,0,1000])
plt.show()